In [114]:
import pickle
import pandas as pd
import spacy 
from presidio_analyzer import AnalyzerEngine
import dataclasses
import re
import nltk


import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('stopwords')

USE_CACHE_DICTS = True # set to False to recompute all dictionaries

# # Read dictionary pkl file
# with open('email_to_departments.pkl', 'rb') as fp:
#     email_to_departments = pickle.load(fp)
#     print('email_to_departments dictionary loaded from pkl file')

[nltk_data] Downloading package stopwords to /Users/maore/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [115]:
enron_df = pd.read_pickle('enron_students.pkl')

In [116]:
# data cleaning
enron_df['To'] = enron_df['To'].fillna('')
enron_df['From'] = enron_df['From'].fillna('')
enron_df['X-From'] = enron_df['X-From'].fillna('')
enron_df['X-To'] = enron_df['X-To'].fillna('')
enron_df['X-cc'] = enron_df['X-cc'].fillna('')
enron_df['X-bcc'] = enron_df['X-bcc'].fillna('')


In [117]:
import re
import pandas as pd

def extract_emails(text):
    if pd.isna(text) or text is None or text == '':
        return []
    # Regex to match email addresses
    email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
    return re.findall(email_pattern, text)

def extract_single_email_from_text(text):
    emails = extract_emails(text)
    if (len(emails) > 1):
        print(f"Multiple emails found in: {text}")
    elif len(emails) == 1:
        return emails[0]
    print(f"Could not extract email from: {text}")
    return None

def extract_sent_to_emails(text):
    emails = extract_emails(text)
    if len(emails) > 0:
        return emails
    return []

def is_between_ect_and_ees(email_to_depart_dict, row, is_including_cc_bcc=False, additional_check=True):
    sender_email = extract_single_email_from_text(row['From'])
    if sender_email in email_to_depart_dict:
        sender_department = email_to_depart_dict[sender_email]
        if sender_department != 'ECT' and sender_department != 'EES':
            return False
        else:
            if is_including_cc_bcc:
                reciever_emails = extract_sent_to_emails(row['To']) + extract_sent_to_emails(row['X-To'])
            else:
                reciever_emails = extract_sent_to_emails(row['To']) + extract_sent_to_emails(row['X-To']) + extract_sent_to_emails(row['X-cc']) + extract_sent_to_emails(row['X-bcc'])
            
            for email in reciever_emails:
                if email in email_to_depart_dict:
                    reciever_department = email_to_depart_dict[email]

                    if sender_department == 'ECT' and reciever_department == 'EES':
                        return True
                    elif sender_department == 'EES' and reciever_department == 'ECT':
                        return True
                    
            if additional_check:
                if 'ECT' in row['X-To']:
                    if 'EES' in row['X-From']:
                        return True
                if 'EES' in row['X-To']:
                    if 'ECT' in row['X-From']:
                        return True
      
            return False
    
    else:
        if additional_check:
            if 'ECT' in row['X-To']:
                if 'EES' in row['X-From']:
                    return True
            if 'EES' in row['X-To']:
                if 'ECT' in row['X-From']:
                    return True
        return False
    


    

In [118]:

def build_dict():
    email_to_departments = {}
    email_to_departments.update({
            'smith.day@enron.com' : 'ECT'
            ,'janet.wallis@enron.com' : 'ECT'
            ,'daren.farmer@enron.com' : 'ECT'
            ,'stuart.zisman@enron.com' : 'ECT'
            ,'melissa.murphy@enron.com' : 'ECT'
            ,'thresa.allen@enron.com' : 'ECT'
            ,'leslie.reeves@enron.com' : 'ECT'
            ,'stacey.white@enron.com' : 'ECT'
            ,'janet.wallis@enron.com' : 'ECT'
            ,'daren.farmer@enron.com' : 'ECT'
            ,'brad.nebergall@enron.com': 'ECT'
            ,'colleen.koenig@enron.com': 'EES'
            ,'john.kinser@enron.com': 'ECT'
            ,'jeff.merola@enron.com': 'EES'
            })
    
    email_to_departments.update({'eric.bass@enron.com': 'ECT'})

    multiple_count = 0
    for index, row in enron_df.iterrows():
        x_emails = extract_emails(row['X-From'])
        from_emails = set([mail.lower() for mail in (extract_emails(row['From']) + x_emails)])
        to_emails = set([mail.lower() for mail in (extract_emails(row['To']) + extract_emails(row['X-To']))])
        if len(from_emails) == 0:
            continue
        if len(from_emails) > 1:
            from_emails = [mail for mail in from_emails if not mail.startswith('imceanotes')]
            if len(from_emails) > 1 or len(from_emails) == 0:
                multiple_count += 1

        for email in from_emails:
            email = email.lower()
            if "enron.com" not in email and email not in email_to_departments:
                email_to_departments.update({email: 'NA'})
            else:
                if email in email_to_departments and email_to_departments[email] != 'Other':
                    continue
                elif 'EES' in row['X-From']:
                    email_to_departments.update({email: 'EES'})
                    if len(to_emails) == 1:
                        email_to_departments.update({list(to_emails)[0]: 'ECT'})
                elif 'ECT' in row['X-From']:
                    email_to_departments.update({email: 'ECT'})
                    if len(to_emails) == 1:
                        email_to_departments.update({list(to_emails)[0]: 'EES'})
                else:
                    email_to_departments.update({email: 'Other'})

        # new addition - need to check
        if len(to_emails) == 1:
            if 'ECT' in row['X-To']:
                email_to_departments.update({list(to_emails)[0]: 'ECT'})
            elif 'EES' in row['X-To']:
                email_to_departments.update({list(to_emails)[0]: 'EES'})

    email_to_departments.update({
            'smith.day@enron.com' : 'ECT'
            ,'janet.wallis@enron.com' : 'ECT'
            ,'daren.farmer@enron.com' : 'ECT'
            ,'stuart.zisman@enron.com' : 'ECT'
            ,'melissa.murphy@enron.com' : 'ECT'
            ,'thresa.allen@enron.com' : 'ECT'
            ,'leslie.reeves@enron.com' : 'ECT'
            ,'stacey.white@enron.com' : 'ECT'
            ,'janet.wallis@enron.com' : 'ECT'
            ,'daren.farmer@enron.com' : 'ECT'
            ,'brad.nebergall@enron.com': 'ECT'
            ,'colleen.koenig@enron.com': 'EES'
            ,'john.kinser@enron.com': 'ECT'
            ,'jeff.merola@enron.com	': 'EES'
            ,'scott.mills@enron.com': 'ECT'
            ,'marilyn.colbert@enron.com': 'ECT'
            ,'molly.harris@enron.com': 'ECT'
            ,'joseph.wagner@enron.com': 'ECT'
            ,'stuart.zisman@enron.com,' : 'ECT'
            ,'janet.wallis@enron.com' : 'ECT'
            ,'daren.farmer@enron.com' : 'ECT'
            ,'stephanie.gardner@enron.com' : 'EES'

            })
    
    return email_to_departments

if USE_CACHE_DICTS:
    email_to_department_dict = pickle.load(open('email_to_departments.pkl', 'rb'))
else:
    email_to_department_dict = build_dict()

In [119]:
len(email_to_department_dict)

20638

In [120]:
enron_df['is_between_ect_and_ees'] = enron_df.apply(lambda row: is_between_ect_and_ees(email_to_department_dict, row), axis=1)

Could not extract email from: u@d.h
Could not extract email from: pep <performance.>


In [121]:
if 'cheryl.dudley@enron.com' in email_to_department_dict:
    print('yes')

yes


In [122]:
# remove all spaces 
enron_df['violated_rules'] = enron_df['violated_rules'].apply(lambda x: x.replace(' ', ''))

# apply split ',' on violated rules
enron_df['violated_rules'] = enron_df['violated_rules'].apply(lambda x: x.split(',')) 

In [123]:
pd.set_option('display.max_colwidth', 1000)
enron_df_not_catched_1_1 = enron_df[(enron_df['is_between_ect_and_ees'] == False) & (enron_df['violated_rules'].apply(lambda x: '1.1' in x ) & (enron_df['X-To'].str.contains('ECT')))]
enron_df_not_catched_1_1.head(10)
# len(enron_df_not_catched_1_1)

# print(is_between_ect_and_ees(test, enron_df_not_catched_1_1.iloc[0], False))


,Date,From,To,X-To,X-From,X-cc,X-bcc,Subject,email_body,verdict,violated_rules,is_between_ect_and_ees
349726,"Wed, 6 Jun 2001 13:03:00 -0700 (PDT)",randal.maffett@enron.com,"don.schroeder@enron.com, patrick.danaher@enron.com, adam.metry@enron.com, \n\tsarah.mulholland@enron.com, bill.briggs@enron.com, \n\tgerald.nemec@enron.com, lisa.vitali@enron.com, bill.white@enron.com, \n\tjames.gough@enron.com, jim.goughary@enron.com, john.wilson@enron.com","Don Schroeder <Don Schroeder/ENRON@enronXgate>, Patrick Danaher <Patrick Danaher/NA/Enron@Enron>, Adam Metry <Adam Metry/ENRON@enronXgate>, Sarah Mulholland <Sarah Mulholland/ENRON@enronXgate>, Bill F Briggs <Bill F Briggs/ENRON@enronXgate>, Gerald Nemec <Gerald Nemec/HOU/ECT@ECT>, Lisa Vitali <Lisa Vitali/ENRON@enronXgate>, White@ <White@/O==ENRON/OU==NA/CN==RECIPIENTS/CN==JWHITE7@EX@enronXgate>, Bill White <Bill White/LON/ECT@ECT>, James Gough <James Gough/Enron@EUEnronXgate>, Jim Goughary <Jim Goughary/ENRON@enronXgate>, John L Wilson <John L Wilson/ENRON@enronXgate>",Randal Maffett <Randal Maffett/ENRON@enronXgate@ENRON>,John L Nowlan <John L Nowlan/ENRON@enronXgate>,,Kurt Wipp trip to Houston,"Kurt Wipp is a recent hire in our London office to focus on origination/mid-market opps. Kurt joins us after 15+ yrs w/ Methanex. I've asked him to spend next week or so in Houston introducing himself to all the traders, originators, finance, RAC, Legal, etc... I have given him most of your names and he will probably be contacting you to try and set up some time to meet. My assistant, Beth Ryan, may also be coordinating some of the meetings. Please take time to visit w/ him, even if only briefly and help him better understand your markets, perspectives, goals and objectives. It's important that in addition to Kurt developing an understanding of our business strategies/opportunities that he also develop and accelerate his ""Enron DNA."" Thanks in advance!",BLOCK,[1.1],False
498426,"Thu, 10 May 2001 12:49:00 -0700 (PDT)",john.kiani@enron.com,v.weldon@enron.com,V Charles Weldon <V Charles Weldon/HOU/ECT@ECT>,John Kiani <John Kiani/ENRON@enronXgate@ENRON>,,,Stagecoach Contract,"Charlie,\n\nHere is the Stagecoach contract that I promised you. Let me know if you have any questions.\n\nRegards,\n\nJohn\n\n",BLOCK,[1.1],False


In [124]:
def update_dict(enron_df_not_catched_1_1, email_to_depart):
    # iterate over the rows of the dataframe enron_df_not_catched_1_1
    updated = 0
    for index, row in enron_df_not_catched_1_1.iterrows():
        x_emails = extract_emails(row['X-From'])
        emails = set([mail.lower() for mail in (extract_emails(row['From']) + x_emails)])
        sent_to_emails = extract_sent_to_emails(row['To']) + extract_sent_to_emails(row['X-To'])
        if len(emails) == 0:
            continue
        if len(emails) > 1:
            emails = [mail for mail in emails if not mail.startswith('imceanotes')]
            if len(emails) > 1 or len(emails) == 0:
                # print(f"From: {row['From']} X-From: {row['X-From']} has multiple emails: {emails}")
                continue
        for email in emails:
            email = email.lower()
            # if from email is in dictionary, get the department
            if email in email_to_depart:
                depart = email_to_depart[email]
                if depart == 'ECT':
                    if len(sent_to_emails) == 1:
                        email_to_depart[sent_to_emails[0]] = 'EES'
                        updated += 1
                elif depart == 'EES':
                    if len(sent_to_emails) == 1:
                        email_to_depart[sent_to_emails[0]] = 'ECT'
                        updated += 1

    print(f"Updated {updated} emails")
    


update_dict(enron_df_not_catched_1_1, email_to_department_dict)


Updated 0 emails


* HERE WE START USER TO LOCATION *


In [125]:
def get_email_to_location_dict(df):
    email_to_location = {}
    for index, row in enron_df.iterrows():
        x_emails = extract_emails(row['X-From'])
        from_emails = set([mail.lower() for mail in (extract_emails(row['From']) + x_emails)])
        if '/OU=EU/' in row['X-From']:
            for email in from_emails:
                email_to_location.update({email: 'EU'})
        elif '/HOU/' in row['X-From']:
            for email in from_emails:
                email_to_location.update({email: 'NA'})
        elif '/OU=NA/' in row['X-From']:
            for email in from_emails:
                email_to_location.update({email: 'NA'})

    # iterate over the dataframe to get 
    violated_rule_1_2 = df[df['violated_rules'].apply(lambda x: '1.2' in x)]

    for index, row in violated_rule_1_2.iterrows():
        from_emails = extract_emails(row['X-From'])
        from_emails = list(set([mail.lower() for mail in (extract_emails(row['From']) + from_emails)]))
        if '/OU=EU/' in row['X-From']:
            for email in from_emails:
                email_to_location.update({email: 'EU'})
        elif '/HOU/' in row['X-From']:
            for email in from_emails:
                email_to_location.update({email: 'NA'})
        elif '/OU=NA/' in row['X-From']:
            for email in from_emails:
                email_to_location.update({email: 'NA'})
        
        sent_to_email = extract_emails(row['X-To'])
        if len(sent_to_email) == 1:
            if from_emails[0] not in email_to_location:
                continue
            if email_to_location[from_emails[0]] == 'EU':
                email_to_location.update({sent_to_email[0]: 'NA'})
            elif email_to_location[from_emails[0]] == 'NA':
                email_to_location.update({sent_to_email[0]: 'EU'})
            
    return email_to_location

if USE_CACHE_DICTS:
    email_to_location = pickle.load(open('email_to_location.pkl', 'rb'))
else:
    email_to_location = get_email_to_location_dict(enron_df)

In [126]:
def is_between_EU_and_NA(row, email_to_location):
    from_email = extract_emails(row['From'])
    from_email = extract_emails(row['X-From']) + from_email
    from_email = list(set([mail.lower() for mail in from_email]))
    to_email = extract_emails(row['To'])
    to_email = extract_emails(row['X-To']) + to_email
    to_email = list(set([mail.lower() for mail in to_email]))
    if len(from_email) == 0 or len(to_email) == 0:
        return False
    if from_email[0] in email_to_location:
        if email_to_location[from_email[0]] == 'EU':
            for email in to_email:
                if email in email_to_location and email_to_location[email] == 'NA':
                    return True
        if email_to_location[from_email[0]] == 'NA':
            for email in to_email:
                if email in email_to_location and email_to_location[email] == 'EU':
                    return True
    
    return False

In [127]:
enron_df['is_EU_To_NA'] = enron_df.apply(lambda row: is_between_EU_and_NA(row, email_to_location), axis=1)

In [128]:
# not_working = enron_df[(enron_df['is_EU_To_NA'] == False) & (enron_df['violated_rules'].apply(lambda x: '1.2' in x))]
# not_working.head(10)

In [129]:
print(len(email_to_location))
# enron_df.head(10)
pd.set_option('display.max_colwidth', 50)

3985


In [130]:
def get_c_suit_dict():
    emails = set({
        'kenneth.lay@enron.com',
        'ken.skilling@enron.com',
        'rbowers@nyiso.com',
        'michael.brown@enron.com',
        'john.sherriff@enron.com',
        'jeffrey.sherrick@enron.com',
        'a..shankman@enron.com',
        'ken.rice@enron.com',
        'greg.piper@enron.com',
        'mark.metts@enron.com',
        'coo.jeff@enron.com',
        'rebecca.mcdonald@enron.com',
        'danny.mccarty@enron.com',
        'dan.leff@enron.com',
        'john.lavorato@enron.com',
        'mark.koenig@enron.com',
        'louise.kitchen@enron.com',
        'stanley.horton@enron.com',
        '40enron@enron.com', # for some reason is tagged for stanley horton
        'ben.glisan@enron.com',
        'mark.frevert@enron.com',
        'andrew.fastow@enron.com',
        'jr..legal@enron.com',
        'derrick@enron.com',
        'david.delainey@enron.com',
        'richard.causey@enron.com',
        'michael.brown@enron.com',
        'raymond.bowen@enron.com',
    })

    violated_rule_1_3 = enron_df[enron_df['violated_rules'].apply(lambda x: '2.2' in x)]

    for index, row in violated_rule_1_3.iterrows():
        from_emails = extract_emails(row['X-From'])
        from_emails = list(set([mail.lower() for mail in (extract_emails(row['From']) + from_emails)]))
        to_email = extract_emails(row['To'])
        to_email = extract_emails(row['X-To']) + to_email
        to_email = list(set([mail.lower() for mail in to_email]))

        if len(from_emails) == 0:
            continue
        for email in from_emails:
            emails.add(email)
            # print(f"Email: {email} is in C-Suit")

        # for email in to_email:
        #     emails.add(email)
        #     print(f"Email: {email} is in C-Suit")

    return emails

if USE_CACHE_DICTS:
    c_suit_emails = pickle.load(open('c_suit_emails.pkl', 'rb'))
else:
    c_suit_emails = get_c_suit_dict()

In [131]:
def is_in_c_suit(row, c_suit_emails):
    # Extract and clean email addresses from 'From' fields
    from_emails = extract_emails(row['X-From']) + extract_emails(row['From'])
    from_emails = list(set(email.lower() for email in from_emails))

    # Extract and clean email addresses from 'To' fields
    to_email = extract_emails(row['X-To']) + extract_emails(row['To'])
    to_email = list(set(email.lower() for email in to_email))

    # Check if both sender and recipient are in the c-suite email list
    return any(email in c_suit_emails for email in from_emails) and any(email in c_suit_emails for email in to_email)



In [132]:
enron_df['is_c_suit'] = enron_df.apply(lambda row: is_in_c_suit(row, c_suit_emails), axis=1)

In [133]:
pd.set_option('display.max_colwidth', 100)

In [134]:
def get_from_to_emails(row):
    from_emails = extract_emails(row['X-From']) + extract_emails(row['From'])
    from_emails = list(set(email.lower() for email in from_emails))

    # Extract and clean email addresses from 'To' fields
    to_email = extract_emails(row['X-To']) + extract_emails(row['To'])
    to_email = list(set(email.lower() for email in to_email))

    return from_emails, to_email

In [135]:
def is_leaving_corporate(row):
    from_emails, to_email = get_from_to_emails(row)
    if len(from_emails) == 0 or len(to_email) == 0:
        return False
    
    for email in to_email:
        if 'enron.com' not in email:
            return True

    return False


def is_leaving_ect(row, email_to_depart):
    from_emails, to_email = get_from_to_emails(row)
    if len(from_emails) == 0 or len(to_email) == 0:
        return False

    for email in from_emails:
        if email in email_to_depart and email_to_depart[email] == 'ECT':
            for t_email in to_email:
                if t_email in email_to_depart and email_to_depart[t_email] != 'ECT':
                    return True

    return False

In [136]:
# save all dictionaries as pickle
import pickle
global USE_CACHE_DICTS
if not USE_CACHE_DICTS:
    pickle.dump(email_to_location, open('email_to_location.pkl', 'wb'))
    pickle.dump(email_to_department_dict, open('email_to_departments.pkl', 'wb'))
    pickle.dump(c_suit_emails, open('c_suit_emails.pkl', 'wb'))
    print('All dictionaries saved as pickle')

In [137]:
class StaticAnalyzer:
    
    @staticmethod
    def is_violating_rule_1_1(row):
        return is_between_ect_and_ees(email_to_department_dict, row, is_including_cc_bcc=False, additional_check=True)
    
    
    @staticmethod
    def is_violating_rule_1_2(row):
        return is_between_EU_and_NA(row, email_to_location)
    
    
    @staticmethod
    def is_violating_rule_1_3(row):
        return is_leaving_corporate(row)
    
    
    @staticmethod
    def is_violating_rule_2_1(row):
        return is_leaving_ect(row, email_to_department_dict)
    
    
    @staticmethod
    def is_violating_rule_2_2(row):
        return is_in_c_suit(row, c_suit_emails)
    
    @staticmethod
    def is_violating_rule_2_3(row):
        return is_leaving_corporate(row)

In [138]:
@dataclasses.dataclass()
class ContentAnalysisResult:
    quids: []
    piis: []

    def __init__(self):
        self.quids = []
        self.piis = []

    def __str__(self):
        return f'(Sensitive: {len(self.piis) > 0 or len(self.quids) > 0} | Quids: {self.quids} | PII: {self.piis})'
    
    def is_sensitive(self, rule_id):
        if rule_id == '2.3':
            return len(self.piis) > 0 or len(self.quids) >= 2
        else:
            return len(self.piis) > 0 or len(self.quids) > 0

@dataclasses.dataclass()
class TopicAnalysisResult:
    is_legal: bool = False
    is_business: bool = False
    is_finance: bool = False
    
    def __str__(self):
        return f'(Legal: {self.is_legal} | Business: {self.is_business} | Finance: {self.is_finance})'
    
@dataclasses.dataclass()
class EnforcerResult:
    violated_rules: []

    def __init__(self):
        self.violated_rules = []

    def is_allowed(self)-> bool:
        return len(self.violated_rules) == 0
    
    def __str__(self):
        return f'Violated Rules: {self.violated_rules}'
    

In [139]:
def find_all_ssn(text):
    return re.findall(r'\d{3}-\d{2}-\d{4}', text)

def find_all_credit_cards(text):
    return re.findall(r'\d{4}-\d{4}-\d{4}-\d{4}', text) 

def find_all_phone_numbers(text):
    return re.findall(r'\(?\d{3}\)?\s*-\s*\d{3}\s*-\s*\d{4}', text) 

def find_sensitive_words(text):
    return re.findall(r'password|attach|confidential', text.lower())


class PII:
    def __init__(self, entity_type, score, text):
        self.entity_type = entity_type
        self.score = score
        self.text = text
        
    def __str__(self):
        return f'(Entity: {self.entity_type} | Score: {self.score} | Text: {self.text})'
        
class ContentAnalyzer:
    
    MAX_DOC_SIZE_FOR_SPACY = 2000
    QUIDS = ['ORG', 'GPE', 'LOW', 'FAC', 'LOC']
    SENSITIVE = ['MONEY', 'PERCENT', 'NORP', 'PRODUCT', 'EVENT']
    POTENTIALLY_SENSITIVE = ['DATE', 'TIME', 'QUANTITY', 'ORDINAL', 'CARDINAL', 'PERSON']
    
    def __init__(self, software='spacy'):      
        self.software = software
        if software == 'spacy':
            self.nlp = spacy.load('en_core_web_sm')
            self.nlp.max_length = 1500000
        elif software == 'presidio':
            self.analyzer = AnalyzerEngine()
        else:
            raise Exception(f'Software {software} is not supported')

        
    def _get_entities(self, document):
        try:
            doc_len = len(document)
            if doc_len > self.MAX_DOC_SIZE_FOR_SPACY:
                document = document[:self.MAX_DOC_SIZE_FOR_SPACY]
            doc = self.nlp(document)
            return [(ent.text, ent.label_) for ent in doc.ents]
    
        except ValueError as e :
            print(f'Error in document. Error: {e}')
            return []
        
        
    def _predict_spacy_verdict(self, row) -> ContentAnalysisResult:
        result = ContentAnalysisResult()
        count_persons = 0
        sensitive_words = find_sensitive_words(row['email_text'])
        if len(sensitive_words) > 0:
            for word in sensitive_words:
                result.piis.append(word)
        if row['quids'] and len(row['quids']) > 0: 
            result.quids = row['quids']
        elif row['sensitive'] and len(row['sensitive']) > 0:
            result.piis.append(row['sensitive'])
        elif row['potentially_sensitive'] and len(row['potentially_sensitive']) > 0:
            for ent in row['potentially_sensitive']:
                if ent[1] == 'DATE':
                    continue
                elif ent[1] == 'TIME':
                    continue
                elif ent[1] == 'QUANTITY':
                    continue
                elif ent[1] == 'ORDINAL':
                    continue
                elif ent[1] == 'CARDINAL':
                    phone_numbers = find_all_phone_numbers(ent[0])
                    credit_cards = find_all_credit_cards(ent[0])
                    ssns = find_all_ssn(ent[0])
                    for phone_number in phone_numbers:
                        result.piis.append(phone_number)
                    for credit_card in credit_cards:
                        result.piis.append(credit_card)
                    for ssn in ssns:
                        result.piis.append(ssn)
                
                #todo maor - think about this, do we want to enter 2 qids in that case? let's see
                # elif ent[1] == 'PERSON':
                #     count_persons += 1
                #     if count_persons > 2:
                #         return 'Sensitive'
                #     continue
        return result
    
    def analyze(self, row: pd.DataFrame) -> ContentAnalysisResult:
        result: ContentAnalysisResult()
        copied_row = row.copy()
        
        if self.software == 'spacy':
            result = self._anaylze_spacy(copied_row)
        elif self.software == 'presidio':
            result = self._analyze_presidio(copied_row)
        else:
            raise Exception(f'Software {self.software} is not supported')
        
        print(f'{self.software} based content analysis result: {result}')
        return result


    def _anaylze_spacy(self, row) -> ContentAnalysisResult:
        row['entities'] = self._get_entities(row['email_text'])
        unique_entities = set()
        for entities in row['entities']:
            for ent in entities:
                if len(ent) < 2:
                    print(f'Entity {ent} is not valid')
                    continue
                unique_entities.add(ent[1])
        row['sensitive'] =  [(ent[0], ent[1]) for ent in row['entities'] if ent[1] in ContentAnalyzer.SENSITIVE]
        row['quids'] = [(ent[0], ent[1]) for ent in row['entities'] if ent[1] in ContentAnalyzer.QUIDS]
        row['potentially_sensitive'] = [(ent[0], ent[1]) for ent in row['entities'] if ent[1] in ContentAnalyzer.POTENTIALLY_SENSITIVE]
        spacy_result = self._predict_spacy_verdict(row)
        
        return spacy_result
    
    NON_SENSITIVE = ['EMAIL_ADDRESS', 'URL', 'ORG']
    SENSITIVE = ['IP_ADDRESS', 'AU_ACN', 'US_ITIN', 'UK_NHS', 'AU_TFN', 'US_BANK_NUMBER', 'IN_PAN', 'US_DRIVER_LICENSE', 'IN_VEHICLE_REGISTRATION', 'SG_NRIC_FIN', 'US_SSN', 'US_PASSPORT', 'MEDICAL_LICENSE', 'PHONE_NUMBER']
    QUASI_SENSITIVE = ['NRP', 'LOCATION', 'PERSON', 'DATE_TIME', 'GPE']
    
    @staticmethod
    def _predict_presidio_verdict(row) -> ContentAnalysisResult:
        result = ContentAnalysisResult()
        
        persons_counter = 0
        if row is None:
            return result
        
        email_text = row['email_text']
        if email_text is None:
            return result
        sensitive_words = find_sensitive_words(row['email_text'])
        phone_numbers = find_all_phone_numbers(email_text)
        credit_cards = find_all_credit_cards(email_text)
        ssns = find_all_ssn(email_text)
        
        for word in sensitive_words:
            result.piis.append(word)
        for phone_number in phone_numbers:
            result.piis.append(phone_number)
        for credit_card in credit_cards:
            result.piis.append(credit_card)
        for ssn in ssns:
            result.piis.append(ssn)
            
    
        if row['pii'] is None:
            return result
        
        for pii in row['pii']:
            if pii.score < 0.5:
                continue
            if pii.entity_type.startswith('IN_'):
                continue
            if pii.entity_type in ContentAnalyzer.NON_SENSITIVE:
                continue
            if pii.entity_type in ContentAnalyzer.SENSITIVE:
                result.piis.append(pii)
            if pii.entity_type in ContentAnalyzer.QUASI_SENSITIVE: 
                # todo maor - think about person
                # if pii.entity_type == 'PERSON':
                #     persons_counter += 1
                #     if persons_counter > 2:
                #         return 'Sensitive'
                # else:
                result.quids.append(pii)
    
            
        return result
    
    def build_pii_text(self, email_body_text, result):
        start = result.start
        end = result.end
        pii_text = email_body_text[start:end]
        return pii_text
        
    
    counter = 0
    def analyze_pii(self, email_body_text):
        global counter
        counter+= 1
        if counter % 200 == 0:
            print(f'Processed {counter} documents')
        pii_list = []
        email_body_text = email_body_text[:2000]
        
        try:
            results = self.analyzer.analyze(text=email_body_text, language='en')
        except Exception as e:
            print(f'Error in document. Error: {e}')
            return []
        
        for result in results:
            pii_text = self.build_pii_text(email_body_text, result)
            pii = PII(result.entity_type, result.score, pii_text)
            pii_list.append(pii)
    
        return pii_list
    


    def _analyze_presidio(self, row) -> ContentAnalysisResult:
        row['pii'] = row['email_text'].apply(lambda x: self.analyze_pii(x))
        presidio_result = self._predict_presidio_verdict(row)
        
        return presidio_result


In [140]:
import preprocessing
import pickle
import os
from gensim.models import LdaMulticore
from gensim.corpora import Dictionary

class TopicAnalyzer:
    def __init__(self):
        self.model: LdaMulticore = LdaMulticore.load(os.path.join('lda', 'lda_model')) 
        self.dictionary: Dictionary = Dictionary.load(os.path.join('lda', 'dictionary'))

        self.rule_to_topic_before =  {
            'None' : [38, 35, 39, 17, 33],
            '1.1' : [9, 35, 7, 20, 29,], # legal
            '1.2': [1, 9, 7, 18, 20, 35, 24] + [5], # financial data
            '1.3': [1, 35, 9, 7, 39] + [5], # business or financial
            '2.1': [1, 9, 7, 35, 24, 20], # financial data
            '2.2': [1, 9, 18, 7, 17, 16, 24] # business data
        }

        self.rule_to_topic =  { 
            'None' : [38, 33, 32, 19, 23, 40, 36, 8, 1, 4],  
            '1.1' : [2, 20, 9, 35, 22, 25, 34, 5, 18, 37, 31, 28, 29, 14, 11], # legal
            '1.2': [9, 34, 6, 11, 18, 37, 22, 16, 20, 35, 10, 7] , # financial data
            '1.3': [35, 9, 20, 12, 23, 18, 39, 31, 4, 24, 10, 7, 28] + [14, 5, 13, 16], # business or financial
            '2.1': [9, 2, 20, 34, 18, 25, 22, 24, 35, 37, 10, 7], # financial data
            '2.2': [9, 18, 34, 20, 17, 24, 28, 16, 31, 39, 10, 7] # business data
        }
        
    def predict(self, row) -> TopicAnalysisResult:
        result = TopicAnalysisResult()
        result.is_legal = self.is_legal(row)
        result.is_business = self.is_business(row)
        result.is_finance = self.is_finance(row)
        
        return result

    
    def is_business(self, row):        
        text = self.process_text(row)
        return self._predict(text, '1.3') | self._predict(text, '2.3')
        
    
    def is_finance(self, row):
        text = self.process_text(row)
        return self._predict(text, '1.2') | self._predict(text, '2.2') | self._predict(text, '1.3')
    
    def is_legal(self, row):
        text = self.process_text(row)
        return self._predict(text, '1.1')
    
    def process_text(self, row):
        subject = row['Subject']
        body = row['email_body']
        text = self._text_to_bow(subject + " " + body)
        return text
    

    def _predict(self, text, label):
        if label not in self.rule_to_topic.keys():
            return False
        
        # clean_text = self._text_to_bow(text, self.dictionary)
        topics_prob = self._get_top_n_probabilities(text, 4, 0.05)
        if topics_prob is None or len(topics_prob) == 0:
            return False
        
        topics = [t[0] for t in topics_prob]
        count = len(self._intersecting_list(topics, self.rule_to_topic[label]))
        count_none = len(self._intersecting_list(topics, self.rule_to_topic['None']))
        if count > 0 and count_none < 3:
            return True
        else:
            return False
        

    def _text_to_bow(self, text):
    # Preprocess the text
        preprocessed_text = preprocessing.preprocess_text(text, should_remove_links=True, should_remove_small_words=True, lemmatize=True, should_remove_digits=True) 
        # Convert the preprocessed text to a bag-of-words using the dictionary
        bow = self.dictionary.doc2bow(preprocessed_text.split())
        return bow

    def _intersecting_list(self, list1, list2):
        return list(set(list1) & set(list2))

    def _get_probability_debug(self, row):
        doc = self._text_to_bow(row['Subject'] + " " + row['email_body'])
        topics =  self._get_top_n_probabilities(doc, 4, 0.08)
        if topics is None:
            return None
        # if empty
        if len(topics) == 0:
            return None
        return [t[0] for t in topics]

    def _get_top_n_probabilities(self, doc, n=5, threshold=0.08):
        if not doc:
            return None  # Handle empty documents
        topic_probs = self.model.get_document_topics(doc, minimum_probability=0)
        if not topic_probs:
            return None  # Handle cases with no significant topic probability
        
        topic_probs = [x for x in topic_probs if x[1] > threshold]
        return sorted(topic_probs, key=lambda x: x[1], reverse=True)[:n]
    

In [143]:
def _only_1_rules_are_violated(static_rule_violations_by_rule_id):
    rule_2_prefix = '2.'
    
    for rule_id in static_rule_violations_by_rule_id.keys():
        if rule_2_prefix in rule_id and static_rule_violations_by_rule_id[rule_id]:
            return False
        
    return True


def only_this_rule_violated(static_rule_violations_by_rule_id, rule_id):
    return static_rule_violations_by_rule_id[rule_id] == True and sum(static_rule_violations_by_rule_id.values()) == 1


def _pre_process(mail_row):
    subject = mail_row['Subject']
    body = mail_row['email_body']
    text = preprocessing.preprocess_text(subject + " " + body, should_remove_small_words=True, should_remove_digits=False, lemmatize=True)
    mail_row['email_text'] = text
    return mail_row


class Enforcer:

    def __init__(self):
        self.static_analyzer = StaticAnalyzer()
        self.topic_analyzer = TopicAnalyzer()
        self.content_analyzer = ContentAnalyzer()

    def enforce(self, mail_row):
        results = []
        topic_analysis = self.topic_analyzer.predict(mail_row.copy())

        if self.static_analyzer.is_violating_rule_1_1(mail_row):
            if topic_analysis.is_legal:
                results.append('1.1')
        if self.static_analyzer.is_violating_rule_1_2(mail_row):
            if topic_analysis.is_finance:
                results.append('1.2')
        if self.static_analyzer.is_violating_rule_1_3(mail_row):
            if topic_analysis.is_business:
                results.append('1.3')

        return results

    def _enforce(self, mail_row) -> EnforcerResult:
        result = EnforcerResult()
        copied_row = mail_row.copy()
        processed_row = _pre_process(copied_row) # not supoose to be in place, but each model should have its own preprocessing!!!
        
        static_rule_violations_by_rule_id: {str : bool} = self.analyze_static(processed_row)
        content_analysis: ContentAnalysisResult
        topic_analysis: TopicAnalysisResult
    
        # static analysis
        if any(static_rule_violations_by_rule_id.values()) is False:
            print(f'After static analysis, The Email is not violating any rule and therefore ALLOWED')
            return result
        
        if only_this_rule_violated(static_rule_violations_by_rule_id, rule_id='2.3'):
            print(f'Only rule 2.3 is potentially violated. therefore, we can skip topic analysis')
            content_analysis: ContentAnalysisResult = self.content_analyzer.analyze(processed_row)
            
            if content_analysis.is_sensitive('2.3'):
                print(f'Email is violating rule 2.3 due to content analysis: {content_analysis} and therefore BLOCKED')
                result.violated_rules.append('2.3')
            else:
                print(f'Email is not violating any rule due to content analysis: {content_analysis} and therefore ALLOWED')
            return result
        
        print(f'Will analyze topic for email, as all rules left required topic analysis')
        topic_analysis = self.topic_analyzer.predict(mail_row.copy())
        
        if _only_1_rules_are_violated(static_rule_violations_by_rule_id):
            content_analysis: ContentAnalysisResult = self.content_analyzer.analyze(processed_row) # todo maor - delete
            print(f'Only Policy 1# rules are violated statically, can skip content analysis')
        else:
            print(f'Will analyze content for , as all rules left required content analysis (2)')
            content_analysis: ContentAnalysisResult = self.content_analyzer.analyze(processed_row)
        
        for rule_id, static_rule_violation in static_rule_violations_by_rule_id.items():
            if static_rule_violation:
                print(f'Email is violating rule {rule_id} due to static analysis. Checking content analysis and topic analysis if needed')
            
                is_violated_rule_by_content_analysis = self._is_violating_rule_by_content(rule_id, content_analysis)
                is_violated_rule_by_topic_analysis = self._is_violating_rule_by_topic(rule_id, topic_analysis)
            
                if is_violated_rule_by_content_analysis and is_violated_rule_by_topic_analysis:
                    print(f'Email is violating rule {rule_id} due to content analysis or topic analysis and therefore BLOCKED')
                    result.violated_rules.append(rule_id)
                else:
                    print(f'Email is not violating rule {rule_id} due to content analysis or topic analysis and therefore ALLOWED')
            else:
                print(f'Email is not violating rule {rule_id} due to static analysis')
        
        return result

    def analyze_static(self, processed_row) -> {str : bool}:
        static_rule_violations_by_rule_id: {str: bool} = {
            '1.1': self.static_analyzer.is_violating_rule_1_1(processed_row),
            '1.2': self.static_analyzer.is_violating_rule_1_2(processed_row),
            '1.3': self.static_analyzer.is_violating_rule_1_3(processed_row),
            '2.1': self.static_analyzer.is_violating_rule_2_1(processed_row),
            '2.2': self.static_analyzer.is_violating_rule_2_2(processed_row),
            '2.3': self.static_analyzer.is_violating_rule_2_3(processed_row)
        }
        
        return static_rule_violations_by_rule_id

    @staticmethod
    def _is_violating_rule_by_content(rule_id, content_analysis: ContentAnalysisResult):
        if '1.' in rule_id:
            return False
        if rule_id == '2.1' or rule_id == '2.2':
            return content_analysis.is_sensitive
        elif rule_id == '2.3':
            return content_analysis.is_sensitive(rule_id)
        else:
            raise Exception(f'Rule {rule_id} is not supported')
        
        
    @staticmethod
    def _is_violating_rule_by_topic(rule_id, topic_analysis: TopicAnalysisResult):
        if rule_id == '1.1':
            return topic_analysis.is_legal
        elif rule_id == '1.2':
            return topic_analysis.is_finance
        elif rule_id == '1.3':
            return topic_analysis.is_finance or topic_analysis.is_business
        elif rule_id == '2.1':
            return topic_analysis.is_finance
        elif rule_id == '2.2':
            return topic_analysis.is_business
        elif rule_id == '2.3':
            return False
        else: 
            raise Exception(f'Rule {rule_id} is not supported for topic analysis')
        
enforcer = Enforcer()

def classify_mail_extended(mail_row) -> EnforcerResult:
    enforcer_results = enforcer._enforce(mail_row)
    return enforcer_results

def classify_mail(mail_row) -> bool:
    enforcer_results = enforcer.enforce(mail_row)
    is_allowed = enforcer_results.is_allowed()
    verdict_prediction = 'ALLOWED' if is_allowed else 'BLOCKED'
    print(f'Email Enforcer results: {enforcer_results}. Will be {verdict_prediction}')
    return is_allowed



In [144]:
topic_analyzer = TopicAnalyzer()


test_df = enron_df[enron_df['violated_rules'].apply(lambda x: '1.3' in x)].sample(frac=0.01, random_state=42)
test_df['is_allowed'] = test_df.apply(lambda row: classify_mail_extended(row), axis=1)

test_df.head(10)

Will analyze topic for email, as all rules left required topic analysis
Will analyze content for , as all rules left required content analysis (2)
spacy based content analysis result: (Sensitive: True | Quids: [('na/enron 02/01/2001', 'ORG'), ('individual.com, inc.', 'ORG'), ('warner isp', 'ORG'), ('time warner', 'ORG'), ('washington', 'GPE'), ('provider earthlink inc.', 'ORG'), ('time warner', 'ORG'), ('cozy relationship', 'ORG'), ('aol time warner', 'ORG'), ('warner cable', 'ORG'), ('atlanta', 'GPE'), ('aol time warner nearly million', 'ORG'), ('warner', 'ORG'), ('aol time warner', 'ORG'), ('federal trade commission federal communication commission', 'ORG'), ('betty', 'ORG')] | PII: ['attach'])
Email is not violating rule 1.1 due to static analysis
Email is not violating rule 1.2 due to static analysis
Email is violating rule 1.3 due to static analysis. Checking content analysis and topic analysis if needed
Email is not violating rule 1.3 due to content analysis or topic analysis and

,Date,From,To,X-To,X-From,X-cc,X-bcc,Subject,email_body,verdict,violated_rules,is_between_ect_and_ees,is_EU_To_NA,is_c_suit,is_allowed
65967,"Thu, 1 Feb 2001 05:25:00 -0800 (PST)",sue.nord@enron.com,"barbara.hueter@enron.com, donald_lassere@enron.net, \n\tgia.maisashvili@enron.com, jeff.dasovich...","Barbara A Hueter, Donald_Lassere@enron.net, Gia Maisashvili, Jeff Dasovich, John Neslage, Lara L...",Sue Nord,,,Individual.com - News From a Friend!,----- Forwarded by Sue Nord/NA/Enron on 02/01/2001 01:24 PM -----\n\n=09apisciotta@kelleydrye.co...,BLOCK,[1.3],False,False,True,Violated Rules: []
164672,"Fri, 12 Oct 2001 07:13:21 -0700 (PDT)",marie.heard@enron.com,mklemont@bear.com,'mklemont@bear.com',"Heard, Marie </O=ENRON/OU=NA/CN=RECIPIENTS/CN=MHEARD>",,,"FW: ECT Europe Finance, Inc. (""ECT Europe"")","\n\n -----Original Message-----\nFrom: =09Heard, Marie =20\nSent:=09Friday, October 12, 2001 9:1...",BLOCK,[1.3],False,False,False,Violated Rules: []
84740,"Thu, 17 Feb 2000 08:18:00 -0800 (PST)",dana.davis@enron.com,kdavis2075@aol.com,kdavis2075@aol.com,Dana Davis,,,This is for daddy. Do Not DELETE!!!,Hwy 290 and I-610 is where this appraisal place for the car is located.\n\nThe physical address ...,BLOCK,[1.3],False,False,False,Violated Rules: []
301913,"Sun, 8 Oct 2000 13:13:00 -0700 (PDT)",kay.mann@enron.com,"nmann@erac.com, suzanne.adams@enron.com","nmann@erac.com, Suzanne Adams",Kay Mann,,,MANN - INV/ITIN & UPGRD,---------------------- Forwarded by Kay Mann/Corp/Enron on 10/08/2000 08:12 \nPM ---------------...,BLOCK,"[1.3, 2.3]",False,False,False,Violated Rules: []
252085,"Wed, 13 Sep 2000 02:44:00 -0700 (PDT)",steven.kean@enron.com,richard@ceg.com,richard@ceg.com,Steven J Kean,,,,"attached is a copy of my california testimony. The proceeding was somewhat \nproductive, but tu...",BLOCK,[1.3],False,False,False,Violated Rules: []
236194,"Fri, 22 Jun 2001 05:09:00 -0700 (PDT)",ursula.brenner@enron.com,rpon@bellatlantic.net,<rpon@bellatlantic.net>,Ursula Brenner,"Steven J Kean, Maureen McVicker",,Steve's presentation - Highlands Forum,"Dick,\n\nThis is the presentation Steve would be using on Monday. I called you to \ncoordinate ...",BLOCK,"[1.3, 2.2, 2.3]",False,False,False,Violated Rules: []
268417,"Fri, 8 Sep 2000 12:01:00 -0700 (PDT)",rosalee.fleming@enron.com,slade_pd@tsu.edu,"""Slade, Priscilla D"" <Slade_PD@tsu.edu>",Rosalee Fleming,,,Re: Kennybunkport and other,Mr. Lay responded that he is unable to attend. I know that you already have \nthat message as s...,BLOCK,[1.3],False,False,False,Violated Rules: []
220107,"Wed, 30 Aug 2000 01:14:00 -0700 (PDT)",shirley.crenshaw@enron.com,wendy.dunford@arthurandersen.com,wendy.dunford@arthurandersen.com @ ENRON,Shirley Crenshaw,Vince J Kaminski,,Re: Vince's London Visit,Hi Wendy:\n\nWe have finally made arrangements for Vince's trip in September. He will\nbe arriv...,BLOCK,[1.3],False,False,False,Violated Rules: []
158512,"Fri, 23 Feb 2001 07:17:00 -0800 (PST)",mary.hain@enron.com,"sbishop@gibbs-bruns.com, jalexander@gibbs-bruns.com","sbishop@gibbs-bruns.com, jalexander@gibbs-bruns.com",Mary Hain,,,Correction to conversation 22 Feb.,"I hope you caught Lysa's error in transcribing this message. I said you \nshould say, ""It doesn...",BLOCK,"[1.3, 2.3]",False,False,False,Violated Rules: []
272813,"Tue, 3 Oct 2000 05:44:00 -0700 (PDT)",katherine.brown@enron.com,"james.bannantine@enron.com, cliff.baxter@enron.com, \n\tsanjay.bhatnagar@enron.com, rick.buy@enr...","James M Bannantine, Cliff Baxter, Sanjay Bhatnagar, Rick Buy, Richard Causey, Diomedes Christodo...",Katherine Brown,"Suzanne Danz, Videoconference@enron, Vanessa Groscrand",,"EXECUTIVE COMMITTEE MEETING - MONDAY, OCTOBER 9","Executive Committee Weekly Meeting\nDate: Monday, October 9\nTime: 11:00 a.m. (CDT)\nLocation:...",BLOCK,"[1.3, 2.3]",False,False,True,Violated Rules: []
